In [2]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.callbacks.manager import CallbackManager
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import ChatGLM
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory
import pickle
from langchain.document_loaders import JSONLoader
import sys
import os
from dotenv import load_dotenv

In [3]:
db_path = "vector_src/mh_src-vectorstore.pkl"
with open(db_path, 'rb') as f:
    db = pickle.load(f)
retriever = VectorStoreRetriever(vectorstore=db)


FileNotFoundError: [Errno 2] No such file or directory: 'vector_src/mh_src-vectorstore.pkl'

In [2]:
from langchain.document_loaders import JSONLoader
question = "根据上面的信息, 写出关于雷电天气下民航飞机的飞行风险及预防措施分析的研究目的, 只写一个段落, 作为论文的一部份.要具体的分析,不能笼统的说,有数据就要引用数据.不要用首先然后这种格式,要有连续性,内容必须丰富,字数1000字"
loader = JSONLoader(
    file_path='out/mh_src.json',
    jq_schema='.[].answer',
)
data = loader.load()
len(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:

!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.0/666.0 kB 2.1 MB/s eta 0:00:0000:0100:01m


In [14]:
len(data[0].page_content)

161

In [4]:
embeddings = HuggingFaceInstructEmbeddings(model_name="../models/stella-large-zh-v2")
vectorstore = FAISS.from_documents(data, embeddings)

No sentence-transformers model found with name ../models/stella-large-zh-v2. Creating a new one with MEAN pooling.


In [60]:
retriever = vectorstore.as_retriever()
tmp_ques = "关于雷电天气下民航飞机的飞行风险及预防措施分析的研究目的"
ds = retriever.get_relevant_documents(query=tmp_ques, top_k=10)
ans = '\n'.join([d.page_content for d in ds])
print(len(ds))
print(ans)

4

 在绪论中，作者杨清华提到了当前航空飞行安全面临的重要性和挑战，特别是在雷雨天气条件下，航空飞行安全面临着更大的风险。这一背景下，保障航空飞行安全成为航空领域关注的焦点。作者还强调了航空气象工作的重要性，以及其在保障航空安全中的关键作用。

这些研究背景为我们理解本文的研究目的和意义提供了重要的线索。通过分析雷雨天气条件下保障航空飞行安全的有效措施，本文旨在探讨在这种特殊环境下如何更好地保障飞行员和乘客的生命安全。这不仅对航空行业具有重要意义，而且对我国航空事业的发展和壮大也具有深远的意义。

此外，作者还提到了一些相关关键词，如“雷雨天气条件下保障航空飞行安全的有效措施”、“中国期刊网”等，为我们进一步了解该领域的最新动态和研究热点提供了参考。总之，在总结和提炼文本内容的基础上，我们可以看出作者的研究背景具有一定的深度和广度，为理解和把握本文的核心内容和观点奠定了基础。

 在绪论中，作者杨清华提到了当前关于飞机飞行影响及安全管控的研究背景。其中，一篇关于雷雨天气条件下保障航空飞行安全的有效措施的研究，以及一篇关于2023年10月的《中国科技信息》杂志上发表了多篇文章，涵盖了计算机网络工程、暖通工程施工、生物化工产品项目、工程造价管理等多个领域。这些背景信息为我们理解本文的研究目的和意义提供了重要的帮助。

首先，飞机飞行的安全和管控是航空业最重要的任务之一。在复杂的天气条件下，如雷雨天气，飞行安全面临着巨大的挑战。因此，研究如何在这样的天气条件下保障飞行安全，对于航空业的发展具有重要意义。其次，《中国科技信息》杂志上的多篇文章涉及了多个领域，展示了我国在这些领域的最新研究成果和技术应用，为读者提供了丰富的知识资源。

综上所述，了解这些研究背景有助于我们更好地理解本文的研究目的和意义。本文可能会探讨雷雨天气条件下保障航空飞行安全的有效措施，并分析这些措施在实际操作中的效果和可行性。同时，文章也可能会关注这些措施在提高航空业整体安全水平方面的作用，为我国航空业的安全发展提供理论支持和实践指导。

 在文章的结语部分，作者对本文的研究方法、结论或观点进行了概括和评价。通过对现有研究的分析，作者指出，飞行安全是航空业的核心问题，而雷雨天气对飞行的影响不容忽视。作者对本文的研究方法、结论或观点进行了详细阐述，强调了小学数学课堂参与式教学在提高学生兴趣和参与度方面的

In [61]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage
from query_data import get_llm
# llm = get_llm('openai')

llm = ChatOpenAI(
    openai_api_base="https://aiapi.xing-yun.cn/v1",
    openai_api_key="sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a",
    temperature=0.0,
    model_name="gpt-4",
    streaming=True,  # ! important
    callbacks=[StreamingStdOutCallbackHandler()]  # ! important
)
QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know".

    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)
out = qa_chain(
    inputs={
        "query": question,
        "contexts": ans,
    }
)

在本研究中，我们将深入探讨雷雨天气条件下航空飞行安全的重要性和挑战，以及如何通过有效的措施来保障飞行员和乘客的生命安全。据中国期刊网的研究显示，雷雨天气会对飞机的电子设备和通信系统产生干扰，从而影响飞机的正常运行。此外，雷雨天气还会导致飞机的发动机进水，从而影响飞机的动力性能。这些都是雷雨天气对航空飞行带来的具体风险。为了应对这些风险，我们需要采取一系列的预防措施。在《教学与研究》2022年13期中，作者们关注了电击对飞行的影响及对策，认为在雷雨天气中，采取正确的操作和预防措施可以有效地降低电击对飞行的影响。这些研究为我们提供了宝贵的参考，但我们还需要进一步研究其他可能影响飞行安全的因素，以及如何通过技术和管理手段来降低这些风险。我们的研究不仅对航空行业具有重要意义，而且对我国航空事业的发展和壮大也具有深远的意义。

In [ ]:
llm = ChatOpenAI(
    openai_api_base="https://flag.smarttrot.com/v1",
    openai_api_key="zk-f93e9d498ec12c0b40e2207cec533e1f",
    temperature=0.0,
    model_name="gpt-3.5-turbo",
    streaming=True,  # ! important
    callbacks=[StreamingStdOutCallbackHandler()]  # ! important
)
llm([HumanMessage(content=question)])

In [56]:
print(out["text"])

根据上述提供的信息，雷雨天气对民航飞机的飞行安全构成了重大的风险。雷雨天气可能导致飞机电子设备和通信系统干扰，影响飞机的正常运行。此外，雷雨天气还可能导致飞机发动机进水，进而影响飞机的动力性能。因此，本研究的目的是通过分析雷雨天气下民航飞机的飞行风险，并提出相应的预防措施，以保障飞行安全。通过引用中国期刊网的研究数据，可以进一步支持和论证飞机在雷雨天气下的飞行风险，并提供相关的预防措施和技术应用。此外，还可以参考《教学与研究》杂志中关于小学数学课堂参与式教学和电击对飞行的影响及对策的研究，以丰富论文的内容和论证。通过深入分析和综合研究，本文旨在为民航飞机在雷雨天气下的飞行风险评估和预防提供理论支持和实践指导。


In [52]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


llm = get_llm('local')
from langchain.schema import HumanMessage

# create messages to be passed to chat LLM
messages = [HumanMessage(content="tell me a long story")]

print(llm(messages))

content="\n Once upon a time, in a land far away, there was a beautiful princess named Rose. She lived in a magnificent castle with her parents, the king and queen, and their loyal subjects. Rose was known throughout the kingdom for her kindness, intelligence, and grace. She was loved by all who knew her, and she spent her days helping those in need and bringing joy to those around her.\n\nOne day, a wicked witch cast a spell on the princess, putting her into a deep sleep. The witch had been jealous of Rose's beauty and wanted to keep her asleep forever. The king and queen were devastated by the news, and they asked their best scientists and doctors to find a way to break the witch's spell.\n\nAfter much research and experimentation, the scientists discovered that the only way to break the spell was to find the witch's crystal ball and destroy it. The king and queen set out on a dangerous journey to find the crystal ball, accompanied by their bravest knights.\n\nAfter many adventures a

In [ ]:
openai_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_base="https://aiapi.xing-yun.cn/v1",
    openai_api_key="sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=0.3
)

messages = [HumanMessage(content="给我讲一个长故事")]

openai_llm(messages)

In [37]:
print(out['text'])

根据上述信息，本文的研究目的是分析雷电天气下民航飞机的飞行风险，并提出相应的预防措施。雷雨天气对航空飞行安全造成了重大威胁，包括对飞机的电子设备和通信系统的干扰，以及发动机进水等问题。为了保障航空飞行的安全，本文将通过对雷电天气下民航飞机飞行风险的深入分析，探讨有效的预防措施。具体而言，本文将通过引用中国期刊网的研究数据，分析雷雨天气对飞机电子设备和通信系统的干扰程度，并提出相应的技术改进和设备保护措施。同时，本文还将探讨雷雨天气对飞机发动机进水的影响机制，并提出相应的预防措施，如改进发动机设计和加强防水措施。此外，本文还将关注飞行员的培训和操作规范，提出在雷电天气下的飞行操作指南，以提高飞行员的应对能力和飞行安全水平。通过这些研究内容，本文旨在为民航飞机在雷电天气下的飞行风险评估和预防提供理论支持和实践指导，为航空业的安全发展做出贡献。


根据上述提供的信息，本文旨在通过分析雷雨天气条件下民航飞机的飞行风险及预防措施，探讨如何更好地保障飞行员和乘客的生命安全。雷雨天气对飞机的电子设备和通信系统产生干扰，可能影响飞机的正常运行；同时，雷雨天气还可能导致飞机的发动机进水，影响飞机的动力性能。因此，本文将通过引用中国期刊网的研究数据，详细分析雷雨天气对民航飞机飞行的具体风险，并提出相应的预防措施。其中，可以引用《教学与研究》2022年13期的研究，探讨小学数学课堂参与式教学的应用实践，以及采取正确的操作和预防措施降低电击对飞行的影响。通过这些研究分析，本文旨在为航空行业提供具体有效的措施，以确保在雷雨天气条件下民航飞机的飞行安全，为航空业的发展和乘客的安全提供理论支持和实践指导。

In [19]:
from langchain.llms import QianfanLLMEndpoint
import os

os.environ["QIANFAN_AK"] = "vt15bg4vtB5usAD4g1dI7EPE"
os.environ["QIANFAN_SK"] = "Zp80nXaC5BoXbDoTRiRiRf6GlkhYYQGg"

llm = QianfanLLMEndpoint(streaming=True)

In [4]:
from query_data import get_llm
from langchain.schema import HumanMessage
llm = get_llm('local')
llm([HumanMessage(content="给我讲一个长故事")])

AIMessage(content='\n 从前，在一个遥远的国度里，有一个美丽的村庄。这个村庄四面环山，风景如画，人们过着和谐安宁的生活。村子里有一位聪明、善良的少年，名叫小明。他乐于助人，因此深受村民们的喜爱。\n\n有一天，村子附近的一座山发生了滑坡，巨石滚下山坡，堵住了村子里唯一的一条河流。这使得村民们生活陷入困境，粮食和水源都面临短缺。面对这种情况，村里的长者们感到非常忧虑，他们决定向国王求助。\n\n国王得知这个消息后，立即派出了一位勇敢的骑士去解决这个难题。骑士历经千辛万苦，终于找到了一位神秘的智者。智者告诉骑士，要解决这个问题，就必须找到那座山上被滚落的巨石。\n\n于是，骑士带着智者的指引，开始了艰苦的攀爬。经过几天的努力，骑士终于成功地找到了那块巨石。他费尽了九牛二虎之力，将巨石搬到了河边。河水恢复了流淌，村民们的生活也得以恢复正常的生产和生活。\n\n为了感谢骑士的英勇行为，国王将村子附近的一片土地赐给了骑士，让他在此建立了一个新的家园。骑士在 new 家园里开始繁殖家禽、种植庄稼，还帮助村民们改善生活条件。渐渐地，这片土地变得肥沃，人们的生活越来越好。\n\n然而，骑士并没有忘记他的初心。他继续关心着村子里的每一个居民，无论老幼。他还常常带领村民们一起劳动，分享他们的收获。在骑士的带领下，这个村子变得更加繁荣昌盛，成为了附近地区的典范。\n\n这个故事告诉我们，一个人的力量是有限的，但只要我们团结一心，共同努力，就能战胜一切困难。同时，我们也要学会珍惜和感恩，用心去关爱身边的人和事。')